In [12]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import pandas_ta as ta
import yfinance as yf
import copy
import random
from pymongo import MongoClient
import sys
from dotenv import load_dotenv
import warnings
import time
from sqlalchemy import create_engine, text

import feedparser
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai

warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

load_dotenv()
mongo_client = MongoClient(os.environ.get("MONGO_URI"))
stock_db = mongo_client["stock_db"]

vsuccess_engine = create_engine(os.environ.get("VSUCCESS_URI"))

In [13]:
def get_article_content(url):

    # Set up headers for the request
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    
    try:
        # Get webpage content
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find article content
        content_div = soup.find("div", {"itemprop": "articleBody", "id": "vst_detail"})
        if not content_div:
            return ""
        
        # Extract text from paragraphs
        article_content = ""
        paragraphs = content_div.find_all("p")
        
        for p in paragraphs:
            # Skip author and publishing info
            if p.get("class") in [["pAuthor"], ["pPublishTimeSource", "right"]]:
                continue
            
            text = p.get_text(strip=True)
            if text:
                article_content += f"{text}\n"
        
        return article_content.strip()
        
    except Exception:
        return ""

def get_summary(content, news_type):
    genai.configure(api_key=os.environ.get("GEMINI_API"))
    model = genai.GenerativeModel('gemini-2.0-flash')

    if news_type == 'tin_doanh_nghiep':
        promt = f"""
            Tóm tắt lại nội dung bài báo sau: {content}. Yêu cầu viết liền thành 1 đoạn văn khoảng 40 từ. 
            Tóm tắt cần được viêt dưới dạng HPG: ... Trong đó HPG là mã cổ phiếu mà bài báo muốn nói tới. Cần trích xuất mã cổ phiếu ra và ghi lên đầu tiên theo dạng như vậy.
            Nêu không có mã cỏ phiếu nào được đề cập thì trình bày thẳng ý chính vào nội dung, tuyệt đối không sử dụng các cụm từ gián tiếp như "Bài báo này, ..." hay "Trong bài báo, ..."
        """
    else:
        promt = f"""
            Tóm tắt lại nội dung bài báo sau: {content}. Yêu cầu viết liền thành 1 đoạn văn khoảng 20 từ. 
            Trình bày thẳng ý chính vào nội dung, tuyệt đối không sử dụng các cụm từ gián tiếp như "Bài báo này, ..." hay "Trong bài báo, ..." .
        """

    response = model.generate_content(promt)
    return response.text

In [14]:
rss_url_dict = {
    'tin_trong_nuoc': {
        'vi_mo': 'https://vietstock.vn/761/kinh-te/vi-mo.rss',
        'kinh_te_dau_tu': 'https://vietstock.vn/768/kinh-te/kinh-te-dau-tu.rss',
        'chinh_sach': 'https://vietstock.vn/143/chung-khoan/chinh-sach.rss',
    },
    'tin_quoc_te': {
        'tai_chinh_quoc_te': 'https://vietstock.vn/772/the-gioi/tai-chinh-quoc-te.rss',
        'chung_khoan_the_gioi': 'https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss',
        'kinh_te_nganh': 'https://vietstock.vn/775/the-gioi/kinh-te-nganh.rss',
    },
    'tin_doanh_nghiep': {
        'hoat_Dong_kinh_doanh': 'https://vietstock.vn/737/doanh-nghiep/hoat-dong-kinh-doanh.rss',
    }
}

In [21]:
news_df_dict = {}
for news_type, url_dict in rss_url_dict.items():
    temp_news_dict = {}

    for url in url_dict.values():
        feed = feedparser.parse(url)
        #lấy 10 tin đầu tiên của mỗi danh mục, riêng tin doanh nghiệp thì lấy 30
        if news_type == 'tin_doanh_nghiep':
            entris_list = feed.entries[:30]
        else:
            entris_list = feed.entries[:10]

        for entry in entris_list:
            title = entry['title']
            content = get_article_content(entry['id'])
            temp_news_dict[title] = {'summary': get_summary(content, news_type), 'link': entry['id']}
            time.sleep(5)

        news_df_dict[news_type] = pd.DataFrame.from_dict(temp_news_dict, orient='index').reset_index().rename(columns={'index':'title'})

In [22]:
news_df_dict['tin_quoc_te'].to_sql('tin_quoc_te', vsuccess_engine, if_exists='replace', index=False)
news_df_dict['tin_trong_nuoc'].to_sql('tin_trong_nuoc', vsuccess_engine, if_exists='replace', index=False)
news_df_dict['tin_doanh_nghiep'].to_sql('tin_doanh_nghiep', vsuccess_engine, if_exists='replace', index=False)

20